# Übung 1 - Simple Recommender System

In der 1. Übung implementieren wir ein einfaches Recommender System, welches nur allgemein gültige Vorschläge macht - sprich keine personalisierte Vorschläge für jeden Benutzer.
Für die Übungen verwenden wir [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset?select=movies_metadata.csv), welcher für die [MovieLens](https://grouplens.org/datasets/movielens/) Rating auch noch Meta-Daten von der [The Movie Database (TMDb)](https://www.themoviedb.org/) der Filme enthält.

In [1]:
import pandas as pd
import warnings; warnings.simplefilter('ignore')

Einlesen der Meta-Daten aller Filme

In [2]:
movies = pd.read_csv('data/movies_all.csv')[['title', 'genres', 'vote_average', 'vote_count']]
movies.head()

,title,genres,vote_average,vote_count
0,Inception,"['Action', 'Thriller', 'Science Fiction', 'Mys...",8.1,14075
1,The Dark Knight,"['Drama', 'Action', 'Crime', 'Thriller']",8.3,12269
2,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",7.2,12114
3,The Avengers (2012),"['Science Fiction', 'Action', 'Adventure']",7.4,12000
4,Deadpool,"['Action', 'Adventure', 'Comedy']",7.4,11444


## 1. Top-Listen

Die einfachste Art die Besten Filme zu finden, ist einfach die Liste nach dem durchschnittlichen Rating zu sortieren und die besten auszugeben.\
Hinweis: [pandas.DataFrame.sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)

In [3]:
# Sortiere die Filme nach `vote_average` und gebe die Top 10 aus.
movies.sort_values('vote_average', ascending=False).head(10)

,title,genres,vote_average,vote_count
40794,Old Fashioned: The Story of the Wisconsin Supp...,"['Documentary', 'History']",10.0,1
42301,Mammal,['Drama'],10.0,1
41716,Dragonfly (1976),['Drama'],10.0,1
42232,Firefly,['Comedy'],10.0,1
40939,Kenny Chesney: Summer In 3D,[],10.0,1
40482,My Future Love,"['Comedy', 'Drama']",10.0,1
40485,Big Jay Oakerson: Live at Webster Hall,['Comedy'],10.0,1
40920,"OMG, I'm a Robot!","['Comedy', 'Science Fiction']",10.0,1
39308,The Danube Exodus,['Documentary'],10.0,1
40892,If God Is Willing and da Creek Don't Rise,['Documentary'],10.0,1


In [4]:
movies.sort_values(['vote_average', 'vote_count'], ascending=False).head(10)

,title,genres,vote_average,vote_count
33494,Avetik,['Drama'],10.0,3
33561,Maidan,['Documentary'],10.0,3
34036,Frankie Boyle: Hurt Like You've Never Been Loved,['Comedy'],10.0,3
35677,Mera Naam Joker,"['Drama', 'Romance']",10.0,3
35686,Butterfly (2015),"['Science Fiction', 'Drama', 'Fantasy', 'Roman...",10.0,3
36354,Redemption,['History'],10.0,2
36715,The Little Hut,"['Comedy', 'Romance']",10.0,2
36842,Survive and Advance,[],10.0,2
36940,LEGO DC Super Hero Girls: Brain Drain,['Animation'],10.0,2
37014,Patient Zero,"['Action', 'Drama', 'Horror', 'Thriller']",10.0,2


Was fällt auf? Es gibt Filme mit nur einer 10er-Bewertung, welche nun zuoberst sind. Das wollen wir natürlich nicht. Auch wenn wir noch zusätzlich nach `vote_count` sortieren, wird es nicht besser.

IMDB verwendet für ihre Top-Charts ein gewichtet Rating, welches folgendermassen definiert ist:

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

* *v* := Anzahl Stimmen
* *m* := Minimum Anzahl Stimmen um auf die Liste zu kommen
* *R* := Durchschnittliches Rating des Filmes
* *C* := Durchschnittliches Rating aller Filme

Lasst uns nun das durchschnittliche Rating aller Filme (C) berechnen und ein Minimum Anzahl Stimmen (m) definieren.

In [5]:
# Berechne das durchschnittle Rating aller Filme
C = movies['vote_average'].mean()
C

5.618217011635836

Hinweis: [pandas.DataFrame.quantile](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html)

In [6]:
# Lasst uns das 95ste Quantile als minimale Anzahl Stimmen definieren. Sprich die Filme auf unsere Top-Liste sollen mehr Stimmen haben, als 95% aller Filme.
m = movies['vote_count'].quantile(0.95)
m

433.90000000000146

Jetzt können wir noch die Funktion für das Gewichtete Rating definieren und anschliessend unsere Top-Liste berechnen.

In [7]:
# Definiere das Gewichtete Rating (x := Zeile des DataFrame)
def weighted_rating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(v+m) * C)

Hinweis: [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)

In [8]:
# Gewichtetes Rating berechnen und einer neuen Spalte zuweisen anschliessend
movies['wr'] = movies.apply(weighted_rating, m=m, C=C, axis=1)

In [9]:
# Variante ohne apply
v = movies['vote_count']
R = movies['vote_average']
movies['wr'] = (v/(v+m) * R) + (m/(v+m) * C)

In [10]:
# sortiere das Dataframe nach dem gewichteten Rating und gebe die Top-20 Filme aus
movies.sort_values('wr', ascending=False).head(20)

,title,genres,vote_average,vote_count,wr
19,The Shawshank Redemption,"['Drama', 'Crime']",8.5,8358,8.357778
61,The Godfather,"['Drama', 'Crime']",8.5,6024,8.306376
1,The Dark Knight,"['Drama', 'Action', 'Crime', 'Thriller']",8.3,12269,8.208397
8,Fight Club,['Drama'],8.3,9678,8.184925
17,Pulp Fiction,"['Thriller', 'Crime']",8.3,8670,8.172184
21,Forrest Gump,"['Comedy', 'Drama', 'Romance']",8.2,8147,8.069450
149,Schindler's List,"['Drama', 'History', 'War']",8.3,4436,8.061058
152,Whiplash,['Drama'],8.3,4376,8.058077
180,Spirited Away,"['Fantasy', 'Adventure', 'Animation', 'Family']",8.3,3968,8.035654
63,The Empire Strikes Back,"['Adventure', 'Action', 'Science Fiction']",8.2,5998,8.025831


Das sieht schon etwas besser au. Jedoch fällt auf das vor allem Dramen und Thriller zuoberst auf der Liste stehen, und keine Romanzen unter die Top-20 geschafft haben.

## Top-Genre-Liste

Darum werden wir im nächsten Schritt nun Top-Listen pro Genre erstellen.\
Hinweis: [pandas.Series.str.contains](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html)

In [11]:
# Implementiere nun folgende Funktion welche für ein bestimmtes Genre die Top-Liste zurückgibt 
def top_genre_chart(movies, genre, percentile=0.85):
    movies_genre = movies[movies.genres.str.contains(genre, regex=False)]
    C = movies_genre['vote_average'].mean()
    m = movies_genre['vote_count'].quantile(percentile)

    movies_genre['wr'] = movies_genre.apply(weighted_rating, args=(m, C), axis=1)
    movies_genre = movies_genre.sort_values('wr', ascending=False).head(250)
    
    return movies_genre

In [12]:
# Gebe nun die Top-20 Romance-Filme aus
top_genre_chart(movies, 'Romance').head(20)

,title,genres,vote_average,vote_count,wr
1635,Dilwale Dulhania Le Jayenge,"['Comedy', 'Drama', 'Romance']",9.1,661,8.701372
1087,Your Name.,"['Romance', 'Animation', 'Drama']",8.5,1030,8.281258
21,Forrest Gump,"['Comedy', 'Drama', 'Romance']",8.2,8147,8.173547
1349,Cinema Paradiso,"['Drama', 'Romance']",8.2,834,7.964387
121,La La Land,"['Comedy', 'Drama', 'Music', 'Romance']",7.9,4745,7.860576
160,Her,"['Romance', 'Science Fiction', 'Drama']",7.9,4215,7.855724
202,Eternal Sunshine of the Spotless Mind,"['Science Fiction', 'Drama', 'Romance']",7.9,3758,7.850467
966,Vertigo,"['Mystery', 'Romance', 'Thriller']",8.0,1162,7.840579
2235,City Lights,"['Comedy', 'Drama', 'Romance']",8.2,444,7.792600
669,Mr. Nobody,"['Science Fiction', 'Drama', 'Romance', 'Fanta...",7.9,1616,7.788307
